In [ ]:
import netCDF4 as nc
import numpy as np
import pickle
import rasterio as rio
import xarray as xr
import rioxarray as rxr
import pandas as pd
from rasterio.transform import rowcol
from zoneinfo import ZoneInfo  


# Loading of the ICON 10m windspeed pickle file
windspeed_10m_icon_table = pd.read_pickle(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\wind-speed-10m_icon_table.pickle")

# Data cleaning(The remove of duplicated and unik value contains columns)
windspeed_10m_icon_table = windspeed_10m_icon_table.loc[:, ~windspeed_10m_icon_table.columns.duplicated()]
windspeed_10m_icon_table = windspeed_10m_icon_table.loc[:, windspeed_10m_icon_table.apply(lambda col: col.nunique() > 1, axis=0)]

# 🔹 Load of the corresponding excel files 
df_mapping_ICON_Station_ID = pd.read_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Station_ID_vs_ICON_code.csv", sep=';')
# 🔹 Verification of columns availability
if not {"Station_ID", "ICON_code"}.issubset(df_mapping_ICON_Station_ID.columns):
    raise ValueError("Les colonnes 'Station_ID' et 'ICON_code' sont absentes du fichier.")
# 🔹 Create a dict of corresponding{ICON_code: Station_ID}
icon_to_station = dict(zip(df_mapping_ICON_Station_ID["ICON_code"].astype(str), df_mapping_ICON_Station_ID["Station_ID"]))
# 🔹 Subtitute the name of the columns instead of index
windspeed_10m_icon_table.rename(columns=icon_to_station, inplace=True)
windspeed_10m_icon_table.index = windspeed_10m_icon_table.index.tz_localize("UTC")

#Loading of the observed 10m windspeed pickle file
obs_10m_table = pd.read_pickle(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\wind-speed-10m_obs_table.pickle")
print("Before shift")
print(obs_10m_table["858148"])

#Set the time zone to UTC
station_SASSCAL_list = [
    "39711", "46943", "47066", "361100", "858246", "21", "22", "23", "24", "25",
    "67581", "67583", "67585", "67591", "67593", "68024", "68026", "68030", "68038",
    "68148", "68151", "68226", "68320", "68325", "68328", "101", "102", "103", "104",
    "105", "106", "108", "109", "110", "111", "112", "113", "114", "115", "116",
    "4230", "4756", "4806", "8893", "31195", "31196", "31197", "31198", "31199",
    "31200", "31202", "31203", "31204", "31205", "31206", "31207", "31209", "31210",
    "31212", "31213", "31214", "31215", "31216", "52121", "58557", "64243", "64258",
    "65934", "65941", "67134", "67135", "74229", "74714", "75823", "80493", "80498",
    "E7624", "E7625", "E7626", "E7627", "E7628", "E7629", "E7630", "E7631",
    "E9126", "E9127", "361096", "361097", "361098", "361099",
    "511942", "511943", "511944",
    "858576", "858577", "858580", "858581", "858583", "858585", "858590", "858596",
    "856134", "858148", "858594"
]
E_cols = ["E7624", "E7625", "E7626", "E7627", "E7628", "E7629", "E7630", "E7631", "E9126", "E9127"]
utc_plus_1_cols = ["46943", "47066", "361100", "858246"]
remains_sasscal_cols = [col for col in station_SASSCAL_list if col not in utc_plus_1_cols and col not in E_cols]
utc_plus_2_cols = [col for col in remains_sasscal_cols if col in obs_10m_table.columns]
remains_obs_all = [col for col in obs_10m_table.columns if col not in utc_plus_1_cols and col not in utc_plus_2_cols]

windspeed_10m_obs_table = pd.DataFrame()

# For station already in UTC
for col in remains_obs_all:
    if col in obs_10m_table.columns:
        s = obs_10m_table[col].copy()
        s.index = s.index.tz_localize("UTC")
        windspeed_10m_obs_table[col] = s

# For station in UTC+1 (like l'Angola)
for col in utc_plus_1_cols:
    if col in obs_10m_table.columns:
        s = obs_10m_table[col].copy()
        s.index = s.index.tz_localize("Africa/Luanda").tz_convert("UTC")
        windspeed_10m_obs_table[col] = s

# For station in UTC+2 (like l'Afrique du Sud)
for col in utc_plus_2_cols:
    if col in obs_10m_table.columns:
        s = obs_10m_table[col].copy()
        s.index = s.index.tz_localize("Africa/Johannesburg").tz_convert("UTC")
        windspeed_10m_obs_table[col] = s      
print("After shift")
print(windspeed_10m_obs_table["858148"])

#End of time zone setting

#The remove of the implausible data
windspeed_10m_obs_table = windspeed_10m_obs_table.where((windspeed_10m_obs_table >= 0) & (windspeed_10m_obs_table <= 40), other=pd.NA)
# 🔹Definition of NaN % limit to delete column
threshold = 0.70
# 🔹 Supprimer les colonnes où PLUS DE `threshold` % des valeurs sont NaN
windspeed_10m_obs_table = windspeed_10m_obs_table.dropna(axis=1, thresh=int((1 - threshold) * len(windspeed_10m_obs_table)))


# ERA5 Start here
stations = pd.read_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Stationlat&lon_new.csv" , sep=';')

ERA5_Wind_2017_10m = nc.Dataset(r"D:\Wascal_2023-2025\Thesis\2025-04-14\wind_speed\43df5441fac8134c5e81bd3aa0c48fb0\data_stream-oper_stepType-instant.nc")
ERA5_Wind_2018_10m = nc.Dataset(r"D:\Wascal_2023-2025\Thesis\2025-04-14\wind_speed\f4cdecdedec5898827f229a7fb114bd5\data_stream-oper_stepType-instant.nc")
ERA5_Wind_2019_10m = nc.Dataset(r"D:\Wascal_2023-2025\Thesis\2025-04-14\wind_speed\c832a1ecc417cc0e2e0a31ebed19cef6\data_stream-oper_stepType-instant.nc")
wind_2017_u10 = ERA5_Wind_2017_10m.variables['u10'][:]
wind_2017_v10 = ERA5_Wind_2017_10m.variables['v10'][:]
wind_2018_u10 = ERA5_Wind_2018_10m.variables['u10'][:]
wind_2018_v10 = ERA5_Wind_2018_10m.variables['v10'][:]
wind_2019_u10 = ERA5_Wind_2019_10m.variables['u10'][:]
wind_2019_v10 = ERA5_Wind_2019_10m.variables['v10'][:]

wind_ERA5_list = []
lats_2017 = ERA5_Wind_2017_10m['latitude'][:]
lons_2017 = ERA5_Wind_2017_10m['longitude'][:]
lats_2018 = ERA5_Wind_2018_10m['latitude'][:]
lons_2018 = ERA5_Wind_2018_10m['longitude'][:]
lats_2019 = ERA5_Wind_2019_10m['latitude'][:]
lons_2019 = ERA5_Wind_2019_10m['longitude'][:]
for _, row in stations.iterrows():
    lat_idx_2017 = np.abs(lats_2017 - row['latitude']).argmin()
    lon_idx_2017 = np.abs(lons_2017 - row['longitude']).argmin()
    lat_idx_2018 = np.abs(lats_2018 - row['latitude']).argmin()
    lon_idx_2018 = np.abs(lons_2018 - row['longitude']).argmin()
    lat_idx_2019 = np.abs(lats_2019 - row['latitude']).argmin()
    lon_idx_2019 = np.abs(lons_2019 - row['longitude']).argmin()
    wind_2017_10m = np.sqrt(((wind_2017_u10[:, lat_idx_2017, lon_idx_2017])**2) + ((wind_2017_v10[:, lat_idx_2017, lon_idx_2017])**2))
    wind_2018_10m = np.sqrt(((wind_2018_u10[:, lat_idx_2018, lon_idx_2018])**2) + ((wind_2018_v10[:, lat_idx_2018, lon_idx_2018])**2))
    wind_2019_10m = np.sqrt(((wind_2019_u10[:, lat_idx_2019, lon_idx_2019])**2) + ((wind_2019_v10[:, lat_idx_2019, lon_idx_2019])**2))
    wind_2017_to_2019_ERA5_10m = np.concatenate([wind_2017_10m, wind_2018_10m, wind_2019_10m], axis=0)
    wind_ERA5_list.append(pd.Series(wind_2017_to_2019_ERA5_10m, name=row['station']))
    
wind_ERA5 = pd.concat(wind_ERA5_list, axis=1)
start_time = "2017-01-01 00:00:00"
wind_ERA5.index = pd.date_range(start=start_time, periods=wind_ERA5.shape[0], freq="h").tz_localize("UTC")
print(wind_ERA5['858148'])

# Extraction of ERA5 WM
WM_Station = ['WM01', 'WM02', 'WM03', 'WM05', 'WM06', 'WM07', 'WM08', 'WM09', 'WM10', 'WM11', 'WM12', 'WM13', 'WM14', 'WM15', 'WM16', 'WM17', 'WM18', 'WM19']
windspeed_10m_ERA5_WM = wind_ERA5[WM_Station]

#ERA5_GWA start here
ERA5_wind_speed_10m_mean_2008to2017_southern_Africa = nc.Dataset(r"D:\Wascal_2023-2025\Thesis\2025-04-14\wind_speed\ERA5_wind_speed_10m_mean_2008to2017_southern_Africa.nc")
lats_ERA5 = ERA5_wind_speed_10m_mean_2008to2017_southern_Africa.variables['latitude'][:]
lons_ERA5 = ERA5_wind_speed_10m_mean_2008to2017_southern_Africa.variables['longitude'][:]
wind_mean_ERA5 = ERA5_wind_speed_10m_mean_2008to2017_southern_Africa.variables['si10'][0, :, :]  # (time, lat, lon)

# === Étape 1 : Load of CSV files of stations carateristics
stations = pd.read_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Stationlat&lon_new.csv" , sep=';')
# === Étape 3 : Nearest grid finding
def get_wind_value(lon, lat):
    lat_idx = np.abs(lats_ERA5 - lat).argmin()
    lon_idx = np.abs(lons_ERA5 - lon).argmin()
    return float(wind_mean_ERA5[lat_idx, lon_idx])  # Convertir en float simple

# === Étape 4 : Application on every stations
stations['wind_mean_10m_ERA5'] = stations.apply(lambda row: get_wind_value(row['longitude'], row['latitude']), axis=1)

# === Load of raster GWA
rds = rxr.open_rasterio(r"C:\Users\LENOVO\Downloads\wind_speed_cog_10m.tif",masked=True).squeeze()

# Use of xarray for nesrest
rds = rds.sortby("y")

# Verification
print("📐 Dimensions raster :", rds.dims)  # ('y', 'x') — bon !
print("🧭 Coordonnées disponibles :", list(rds.coords))  # ['x', 'y']

# === Nearest methods
interpolated = rds.interp(
    x=("points", stations["longitude"].values),
    y=("points", stations["latitude"].values),
    method="nearest"
)

# === Ajouter la colonne au tableau
stations["wind_mean_10m_GWA_xarray_nearest"] = interpolated.values

stations.to_excel(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Stations_ERA5_GWA_mean_xarray_nearest.xlsx", index=False)
stations['scaling_factor'] = stations['wind_mean_10m_GWA_xarray_nearest'] / stations['wind_mean_10m_ERA5']
stations.to_excel(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Station_scaling_ERA5_GWA_array_nearest.xlsx", index=False)
print("✅ Nearest methode avec xarray terminée !")        

wind_ERA5_GWA_list = []

for _, row in stations.iterrows():
    station_code = row['station']
    if station_code in wind_ERA5.columns:
        wind_ts = wind_ERA5[station_code]
        wind_correction = wind_ts * row['scaling_factor']
        wind_ERA5_GWA_list.append(pd.Series(wind_correction, name=station_code))

# Fusionner les séries en un DataFrame
wind_ERA5_GWA = pd.concat(wind_ERA5_GWA_list, axis=1)

# Ajouter une index temporel
start_time = "2017-01-01 00:00:00"
wind_ERA5_GWA.index = pd.date_range(start=start_time, periods=wind_ERA5_GWA.shape[0], freq="h").tz_localize("UTC")
print(wind_ERA5_GWA['858148'])
#extraire les WM de ERA5
windspeed_10m_ERA5_GWA_WM = wind_ERA5_GWA[WM_Station]

# 🔹 Find identic column
common_stations = list(set(windspeed_10m_obs_table.columns) & set(windspeed_10m_icon_table.columns) & set(wind_ERA5.columns) & set(wind_ERA5_GWA.columns))
# 🔹 Filtrer les DataFrames pour ne garder que les stations communes
windspeed_10m_obs_table = windspeed_10m_obs_table[common_stations]
windspeed_10m_icon_table = windspeed_10m_icon_table[common_stations]
windspeed_10m_ERA5_table = wind_ERA5[common_stations]
windspeed_10m_ERA5_GWA_table = wind_ERA5_GWA[common_stations]



In [ ]:
#  ICON-LAM and Observed data Dict creation
obs_dict = {}   # Contiendra les données d'observation (WS_10_obs)
icon_dict = {}  # Contiendra les données simulées ICON (WS_10_ICON)

#  Conversion of dataframe into dict
for station in windspeed_10m_obs_table.columns:
    obs_dict[station] = pd.DataFrame({"WS_10_obs": windspeed_10m_obs_table[station]})

for station in windspeed_10m_icon_table.columns:
    icon_dict[station] = pd.DataFrame({"WS_10_ICON": windspeed_10m_icon_table[station]})

#  Station information loading
station_info = pd.read_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Station_ID_vs_ICON_code & lat_lon_Label.csv", sep=';')

required_columns = {"Station_ID", "longitude", "latitude", "label"}
if not required_columns.issubset(station_info.columns):
    raise ValueError("Les colonnes 'Station_ID', 'longitude', 'latitude', 'label' sont absentes du fichier CSV.")

station_meta = station_info.set_index("Station_ID")[["longitude", "latitude", "label"]].to_dict(orient="index")

# dict configuration
for station in obs_dict.keys():
    if station in station_meta:
        obs_dict[station]["Station_ID"] = station
        obs_dict[station]["longitude"] = station_meta[station]["longitude"]
        obs_dict[station]["latitude"] = station_meta[station]["latitude"]
        obs_dict[station]["label"] = station_meta[station]["label"]

for station in icon_dict.keys():
    if station in station_meta:
        icon_dict[station]["Station_ID"] = station
        icon_dict[station]["longitude"] = station_meta[station]["longitude"]
        icon_dict[station]["latitude"] = station_meta[station]["latitude"]
        icon_dict[station]["label"] = station_meta[station]["label"]


# ERA5 and ERA5_GWA dict creation
ERA5_dict = {}   # Contiendra les données d'observation (WS_10_ERA5)
ERA5_GWA_dict = {}   # Contiendra les données d'observation (WS_10_ERA5_GWA)
# 🔹 Convertir les DataFrames en dictionnaires basés sur les colonnes actuelles (stations)
for station in windspeed_10m_ERA5_table.columns:
    ERA5_dict[station] = pd.DataFrame({"WS_10_ERA5": windspeed_10m_ERA5_table[station]})

for station in windspeed_10m_ERA5_GWA_table.columns:
    ERA5_GWA_dict[station] = pd.DataFrame({"WS_10_ERA5_GWA": windspeed_10m_ERA5_GWA_table[station]})

# 🔹 Ajouter les métadonnées à chaque station dans les dictionnaires
for station in ERA5_dict.keys():
    if station in station_meta:
        ERA5_dict[station]["Station_ID"] = station
        ERA5_dict[station]["longitude"] = station_meta[station]["longitude"]
        ERA5_dict[station]["latitude"] = station_meta[station]["latitude"]
        ERA5_dict[station]["label"] = station_meta[station]["label"]
        
for station in ERA5_GWA_dict.keys():
    if station in station_meta:
        ERA5_GWA_dict[station]["Station_ID"] = station
        ERA5_GWA_dict[station]["longitude"] = station_meta[station]["longitude"]
        ERA5_GWA_dict[station]["latitude"] = station_meta[station]["latitude"]
        ERA5_GWA_dict[station]["label"] = station_meta[station]["label"]



In [ ]:
# 🔹 Initialiser un dictionnaire pour stocker les résultats ICON
metrics_ICON_dict = {}

# 🔹 Calculer les métriques pour chaque station
for station in obs_dict.keys():
    if station in icon_dict:  # Vérifier que la station est présente dans les deux dictionnaires
        obs_data = obs_dict[station]["WS_10_obs"].dropna()  # Supprimer les NaN
        icon_data = icon_dict[station]["WS_10_ICON"].dropna()  # Supprimer les NaN

        # 🔹 Aligner les données en fonction de l'index (temps)
        obs_data.name = "WS_10_obs"
        icon_data.name = "WS_10_ICON"
        merged_df_ICON = pd.concat([obs_data, icon_data], axis=1, join="inner")

        if not merged_df_ICON.empty:
            mae = np.mean(np.abs(merged_df_ICON["WS_10_ICON"] - merged_df_ICON["WS_10_obs"]))  # Mean Absolute Error
            me = np.mean(merged_df_ICON["WS_10_ICON"] - merged_df_ICON["WS_10_obs"])  # Mean Error
                        
            # 🔹 Pearson r (calculé manuellement selon la formule souhaitée)
            O = merged_df_ICON["WS_10_obs"].values
            M = merged_df_ICON["WS_10_ICON"].values  # 2ᵉ colonne du modèle (ICON, ERA5 ou ERA5_GWA)

            O_bar = np.mean(O)
            M_bar = np.mean(M)
            numerateur = np.sum((M - M_bar) * (O - O_bar))
            denominateur = np.sqrt(np.sum((M - M_bar)**2)) * np.sqrt(np.sum((O - O_bar)**2))
            pearson_r = numerateur / denominateur if denominateur != 0 else np.nan
            
            
            hist_obs, bin_edges = np.histogram(merged_df_ICON["WS_10_obs"], bins=20, density=True)
            hist_icon, _ = np.histogram(merged_df_ICON["WS_10_ICON"], bins=bin_edges, density=True)
            pss = np.sum(np.minimum(hist_obs, hist_icon)) * np.diff(bin_edges).mean()

            # 🔹 Stocker les résultats
            metrics_ICON_dict[station] = {
                "Station_ID": station,
                "longitude": obs_dict[station]["longitude"].iloc[0],
                "latitude": obs_dict[station]["latitude"].iloc[0],
                "label": obs_dict[station]["label"].iloc[0],
                "MAE": mae,
                "ME": me,
                "Pearson r": pearson_r,
                "PSS": pss
            }

# 🔹 Convertir en DataFrame pour affichage et sauvegarde
metrics_ICON_df = pd.DataFrame.from_dict(metrics_ICON_dict, orient="index")
# 🔹 Sauvegarder le DataFrame en CSV pour utilisation future (optionnel)
metrics_ICON_df.to_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ICON_LAM_results.csv", index=False)


# 🔹 Initialiser un dictionnaire pour stocker les résultats ERA5
metrics_ERA5_dict = {}

# 🔹 Calculer les métriques pour chaque station
for station in obs_dict.keys():
    if station in ERA5_dict:  # Vérifier que la station est présente dans les deux dictionnaires
        obs_data = obs_dict[station]["WS_10_obs"].dropna()  # Supprimer les NaN
        ERA5_data = ERA5_dict[station]["WS_10_ERA5"].dropna()  # Supprimer les NaN

        # 🔹 Aligner les données en fonction de l'index (temps)
        obs_data.name = "WS_10_obs"
        ERA5_data.name = "WS_10_ERA5"
        merged_df_ERA5 = pd.concat([obs_data, ERA5_data], axis=1, join="inner")

        if not merged_df_ERA5.empty:
            mae = np.mean(np.abs(merged_df_ERA5["WS_10_ERA5"] - merged_df_ERA5["WS_10_obs"]))  # Mean Absolute Error
            me = np.mean(merged_df_ERA5["WS_10_ERA5"] - merged_df_ERA5["WS_10_obs"])  # Mean Error
            
            # 🔹 Pearson r (calculé manuellement selon la formule souhaitée)
            O = merged_df_ERA5["WS_10_obs"].values
            M = merged_df_ERA5["WS_10_ERA5"].values  # 2ᵉ colonne du modèle (ICON, ERA5 ou ERA5_GWA)

            O_bar = np.mean(O)
            M_bar = np.mean(M)
            numerateur = np.sum((M - M_bar) * (O - O_bar))
            denominateur = np.sqrt(np.sum((M - M_bar)**2)) * np.sqrt(np.sum((O - O_bar)**2))
            pearson_r = numerateur / denominateur if denominateur != 0 else np.nan

            
            hist_obs, bin_edges = np.histogram(merged_df_ERA5["WS_10_obs"], bins=20, density=True)
            hist_icon, _ = np.histogram(merged_df_ERA5["WS_10_ERA5"], bins=bin_edges, density=True)
            pss = np.sum(np.minimum(hist_obs, hist_icon)) * np.diff(bin_edges).mean()

            # 🔹 Stocker les résultats
            metrics_ERA5_dict[station] = {
                "Station_ID": station,
                "longitude": obs_dict[station]["longitude"].iloc[0],
                "latitude": obs_dict[station]["latitude"].iloc[0],
                "label": obs_dict[station]["label"].iloc[0],
                "MAE": mae,
                "ME": me,
                "Pearson r": pearson_r,
                "PSS": pss
            }

# 🔹 Convertir en DataFrame pour affichage et sauvegarde
metrics_ERA5_df = pd.DataFrame.from_dict(metrics_ERA5_dict, orient="index")
# 🔹 Sauvegarder le DataFrame en CSV pour utilisation future (optionnel)
metrics_ERA5_df.to_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ERA5_results.csv", index=False)


# 🔹 Initialiser un dictionnaire pour stocker les résultats ERA5_GWA
metrics_ERA5_GWA_dict = {}

# 🔹 Calculer les métriques pour chaque station
for station in obs_dict.keys():
    if station in ERA5_GWA_dict:  # Vérifier que la station est présente dans les deux dictionnaires
        obs_data = obs_dict[station]["WS_10_obs"].dropna()  # Supprimer les NaN
        ERA5_GWA_data = ERA5_GWA_dict[station]["WS_10_ERA5_GWA"].dropna()  # Supprimer les NaN

        # 🔹 Aligner les données en fonction de l'index (temps)
        obs_data.name = "WS_10_obs"
        ERA5_GWA_data.name = "WS_10_ERA5_GWA"
        merged_df_ERA5_GWA = pd.concat([obs_data, ERA5_GWA_data], axis=1, join="inner")

        if not merged_df_ERA5.empty:
            mae = np.mean(np.abs(merged_df_ERA5_GWA["WS_10_ERA5_GWA"] - merged_df_ERA5_GWA["WS_10_obs"]))  # Mean Absolute Error
            me = np.mean(merged_df_ERA5_GWA["WS_10_ERA5_GWA"] - merged_df_ERA5_GWA["WS_10_obs"])  # Mean Error
            
            # 🔹 Pearson r (calculé manuellement selon la formule souhaitée)
            O = merged_df_ERA5_GWA["WS_10_obs"].values
            M = merged_df_ERA5_GWA["WS_10_ERA5_GWA"].values  # 2ᵉ colonne du modèle (ICON, ERA5 ou ERA5_GWA)

            O_bar = np.mean(O)
            M_bar = np.mean(M)
            numerateur = np.sum((M - M_bar) * (O - O_bar))
            denominateur = np.sqrt(np.sum((M - M_bar)**2)) * np.sqrt(np.sum((O - O_bar)**2))
            pearson_r = numerateur / denominateur if denominateur != 0 else np.nan

                        
            hist_obs, bin_edges = np.histogram(merged_df_ERA5_GWA["WS_10_obs"], bins=20, density=True)
            hist_ERA5_GWA, _ = np.histogram(merged_df_ERA5_GWA["WS_10_ERA5_GWA"], bins=bin_edges, density=True)
            pss = np.sum(np.minimum(hist_obs, hist_ERA5_GWA)) * np.diff(bin_edges).mean()

            # 🔹 Stocker les résultats
            metrics_ERA5_GWA_dict[station] = {
                "Station_ID": station,
                "longitude": obs_dict[station]["longitude"].iloc[0],
                "latitude": obs_dict[station]["latitude"].iloc[0],
                "label": obs_dict[station]["label"].iloc[0],
                "MAE": mae,
                "ME": me,
                "Pearson r": pearson_r,
                "PSS": pss
            }

# 🔹 Convertir en DataFrame pour affichage et sauvegarde
metrics_ERA5_GWA_df = pd.DataFrame.from_dict(metrics_ERA5_GWA_dict, orient="index")
# 🔹 Sauvegarder le DataFrame en CSV pour utilisation future (optionnel)
metrics_ERA5_GWA_df.to_csv(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ERA5_GWA_results.csv", index=False)
metrics_ERA5_GWA_df.to_excel(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ERA5_GWA_results.xlsx", index=False)
metrics_ERA5_df.to_excel(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ERA5_results.xlsx", index=False)
metrics_ICON_df.to_excel(r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ICON_LAM_results.xlsx", index=False)

print("Metrics ICON_LAM")
print(metrics_ICON_df.head())
print("Metrics ERA5")
print(metrics_ERA5_df.head())
print("Metrics ERA5_GWA")
print(metrics_ERA5_GWA_df.head())

In [ ]:
def print_extremes(df, dataset_name):
    print(f"\n📊 Dataset: {dataset_name}")
    metrics = ["MAE", "ME", "Pearson r", "PSS"]

    for metric in metrics:
        print(f"\n🔹 Metric: {metric}")

        # Max value
        max_row = df.loc[df[metric].idxmax()]
        print(f"⬆️ Highest {metric}:")
        print(max_row.to_string())

        # Min value
        min_row = df.loc[df[metric].idxmin()]
        print(f"\n⬇️ Lowest {metric}:")
        print(min_row.to_string())

# 📥 Apply to each dataset
print_extremes(metrics_ICON_df, "ICON-LAM")
print_extremes(metrics_ERA5_df, "ERA5")
print_extremes(metrics_ERA5_GWA_df, "ERA5_GWA")



def count_mae_intervals(df, dataset_name):
    print(f"\n📊 MAE Intervals for {dataset_name}")

    # Définir les intervalles
    bins = [0, 1, 2, 4, float('inf')]
    labels = ["[0–1]", "(1–2]", "(2–4]", ">4"]

    # Créer une nouvelle colonne avec les intervalles
    df["MAE_interval"] = pd.cut(df["MAE"], bins=bins, labels=labels, right=True)

    # Compter les occurrences par intervalle
    counts = df["MAE_interval"].value_counts().sort_index()

    # Afficher les résultats
    for label in labels:
        print(f"{label}: {counts.get(label, 0)} stations")

# 📥 Appliquer à chaque dataset
count_mae_intervals(metrics_ICON_df, "ICON-LAM")
count_mae_intervals(metrics_ERA5_df, "ERA5")
count_mae_intervals(metrics_ERA5_GWA_df, "ERA5_GWA")



In [ ]:
# 🔹 Liste des DataFrames à traiter
metrics_dataset_dfs = {
    "ICON-LAM": metrics_ICON_df,
    "ERA5": metrics_ERA5_df,
    "ERA5_GWA": metrics_ERA5_GWA_df
}

# 🔹 Colonnes à moyenner
metric_columns = ["MAE", "ME", "Pearson r", "PSS"]

# 🔹 Initialiser un tableau de résultats
summary_means_dataset = pd.DataFrame(columns=metric_columns)

# 🔹 Calculer la moyenne pour chaque jeu de données
for name, df in metrics_dataset_dfs.items():
    summary_means_dataset.loc[name] = df[metric_columns].mean()

# 🔹 Afficher le résumé des moyennes
print("📊 Metric Average from each dataset  :")
print(summary_means_dataset)

# 🔹 Définir les quantiles que tu veux
quantiles = [0.10, 0.20, 0.25, 0.30, 0.40, 0.50, 0.60, 0.70, 0.75, 0.80, 0.90, 0.95]

# 🔹 Colonnes des métriques
metrics = ["MAE", "ME", "Pearson r", "PSS"]

# 🔹 Calculer les quantiles globalement (sur toutes les stations)
quantile_ICON_df = metrics_ICON_df[metrics].quantile(quantiles)

# 🔹 Nettoyer l'affichage
quantile_ICON_df.index = [f"{int(q*100)}%" for q in quantiles]
quantile_ICON_df.index.name = "Quantile"

# 🔹 Afficher le tableau
print("📊 Quantiles globaux pour les métriques ICON-LAM :")
print(quantile_ICON_df)


# 🔹 Colonnes des métriques
metrics = ["MAE", "ME", "Pearson r", "PSS"]

# 🔹 Calculer les quantiles globalement (sur toutes les stations)
quantile_ERA5_df = metrics_ERA5_df[metrics].quantile(quantiles)

# 🔹 Nettoyer l'affichage
quantile_ERA5_df.index = [f"{int(q*100)}%" for q in quantiles]
quantile_ERA5_df.index.name = "Quantile"

# 🔹 Afficher le tableau
print("📊 Quantiles globaux pour les métriques ERA5 :")
print(quantile_ERA5_df)


# 🔹 Colonnes des métriques
metrics = ["MAE", "ME", "Pearson r", "PSS"]

# 🔹 Calculer les quantiles globalement (sur toutes les stations)
quantile_ERA5_GWA_df = metrics_ERA5_GWA_df[metrics].quantile(quantiles)

# 🔹 Nettoyer l'affichage
quantile_ERA5_GWA_df.index = [f"{int(q*100)}%" for q in quantiles]
quantile_ERA5_GWA_df.index.name = "Quantile"

# 🔹 Afficher le tableau
print("📊 Quantiles globaux pour les métriques ERA5_GWA :")
print(quantile_ERA5_GWA_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib as mpl
import matplotlib.colors as mcolors

# === Chargement des données ===
metrics_files = {
    "ICON-LAM": r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ICON_LAM_results.csv",
    "ERA5": r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ERA5_results.csv",
    "ERA5_GWA": r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\Metrics\metrics_10m_ERA5_GWA_results.csv"
}
metrics_data = {name: pd.read_csv(path) for name, path in metrics_files.items()}
station_shapes = {"SASSCALWN": "s", "TAHMO": "^", "NCEI": "o"}

world = gpd.read_file("https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_countries.zip")
africa_south = world[world["NAME"].isin([
    "South Africa", "Namibia", "Botswana", "Zimbabwe", "Mozambique",
    "Angola", "Lesotho", "Eswatini", "Zambia"
])]

# === Métrique ciblée
metric = "MAE"
title = "Mean Absolute Error (MAE)"
# Plages de valeurs
bounds = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
n_colors = len(bounds) - 1

# Couleurs correspondantes (autant que d'intervalles)
custom_colors = ['#253494', '#41b6c4', '#ffffcc',
                 '#fed976', '#f03b20', '#bd0026'][:n_colors]

# Colormap personnalisée
cmap = mcolors.ListedColormap(custom_colors)

# === Récupérer toutes les valeurs (pour une échelle cohérente)
all_values = []


for df in metrics_data.values():
    all_values.extend(df[metric].dropna().values)
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=cmap.N)

# === Création des sous-cartes
fig, axes = plt.subplots(1, 3, figsize=(22, 20), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0.12, bottom=0.4, top=0.98)

for ax, (name, df) in zip(axes, metrics_data.items()):
    africa_south.plot(ax=ax, color="whitesmoke", edgecolor="black")
    for label, shape in station_shapes.items():
        subset = df[df["label"] == label]
        ax.scatter(
            subset["longitude"], subset["latitude"],
            c=subset[metric], cmap=cmap, norm=norm,
            s=100, marker=shape, edgecolor="black"
        )
        # 🔁 Graduation personnalisée pour latitude
    lat_ticks = [-42, -35, -30, -25, -20, -15, -10]  # Valeurs en °S
    lat_labels = [f"{abs(val)}°S" for val in lat_ticks]
        # 🔁 Graduation personnalisée pour longitude
    lon_ticks = [10, 15, 20, 25, 30, 35, 40]  # Valeurs en °E
    lon_labels = [f"{val}°E" for val in lon_ticks]

    
    ax.set_title(f"{name}", fontsize=20)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    ax.set_xticklabels(lon_labels, fontsize=14)
    ax.set_yticklabels(lat_labels, fontsize=14)
    # Limiter l'affichage à la zone géographique ciblée
    ax.set_xlim(min(lon_ticks), max(lon_ticks))
    ax.set_ylim(min(lat_ticks), max(lat_ticks))
    ax.set_xlabel("Longitude", fontsize=16)
    ax.set_ylabel("Latitude", fontsize=16)

# ✅ Barre de couleurs
cbar_ax = fig.add_axes([0.2, 0.48, 0.5, 0.02])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                    cax=cbar_ax, orientation='horizontal', extend='max')
cbar.set_label("MAE in [m/s]", loc='right', fontsize=16)
cbar.ax.xaxis.set_label_position('top') 
cbar.ax.tick_params(labelsize=14)  # ou 16, 18 selon ton besoin

    # 🔹 Légende des réseaux
legend_elements = [plt.Line2D([0], [0], marker=station_shapes[lab], color='black',
                                  linestyle='None', label=lab, markersize=14,
                                  markerfacecolor='white', markeredgecolor='black')
                       for lab in station_shapes]
ax.legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=14)


# Sauvegarde
save_path = r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\Compared_Dataset_MAE_10m.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()





# === Métrique ciblée
metric = "ME"
title = "Mean  Error (ME)"
# Plages de valeurs
bounds = [-4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0,]
n_colors = len(bounds) - 1

# Couleurs correspondantes (autant que d'intervalles)
custom_colors = ['#253494', '#2c7fb8', '#41b6c4', '#a1dab4', '#ffffcc',
                 '#fed976', '#feb24c', '#f03b20', '#bd0026'][:n_colors]
# Colormap personnalisée
cmap = plt.get_cmap('RdBu_r')

# === Récupérer toutes les valeurs (pour une échelle cohérente)
all_values = []


for df in metrics_data.values():
    all_values.extend(df[metric].dropna().values)
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=cmap.N)

# === Création des sous-cartes
fig, axes = plt.subplots(1, 3, figsize=(22, 20), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0.12, bottom=0.4, top=0.98)

for ax, (name, df) in zip(axes, metrics_data.items()):
    africa_south.plot(ax=ax, color="whitesmoke", edgecolor="black")
    for label, shape in station_shapes.items():
        subset = df[df["label"] == label]
        ax.scatter(
            subset["longitude"], subset["latitude"],
            c=subset[metric], cmap=cmap, norm=norm,
            s=100, marker=shape, edgecolor="black"
        )
        # 🔁 Graduation personnalisée pour latitude
    lat_ticks = [-42, -35, -30, -25, -20, -15, -10]  # Valeurs en °S
    lat_labels = [f"{abs(val)}°S" for val in lat_ticks]
        # 🔁 Graduation personnalisée pour longitude
    lon_ticks = [10, 15, 20, 25, 30, 35, 40]  # Valeurs en °E
    lon_labels = [f"{val}°E" for val in lon_ticks]

    
    ax.set_title(f"{name}", fontsize=20)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    ax.set_xticklabels(lon_labels, fontsize=14)
    ax.set_yticklabels(lat_labels, fontsize=14)
    # Limiter l'affichage à la zone géographique ciblée
    ax.set_xlim(min(lon_ticks), max(lon_ticks))
    ax.set_ylim(min(lat_ticks), max(lat_ticks))
    ax.set_xlabel("Longitude", fontsize=16)
    ax.set_ylabel("Latitude", fontsize=16)

# ✅ Barre de couleurs
cbar_ax = fig.add_axes([0.2, 0.48, 0.5, 0.02])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                    cax=cbar_ax, orientation='horizontal', extend = 'both')
cbar.set_label("ME in [m/s]", loc='right', fontsize=16)
cbar.ax.xaxis.set_label_position('top') 
cbar.ax.tick_params(labelsize=14)  # ou 16, 18 selon ton besoin




    # 🔹 Légende des réseaux
legend_elements = [plt.Line2D([0], [0], marker=station_shapes[lab], color='black',
                                  linestyle='None', label=lab, markersize=14,
                                  markerfacecolor='white', markeredgecolor='black')
                       for lab in station_shapes]
ax.legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=14)


# Sauvegarde
save_path = r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\Compared_Dataset_ME_10m.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()




# === Métrique ciblée
metric = "Pearson r"
title = "Pearson Correlation (r)"
# Plages de valeurs
bounds = [0.0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
n_colors = len(bounds) - 1

# Couleurs correspondantes (autant que d'intervalles)
custom_colors = ['#bd0026', '#f03b20', '#feb24c', '#fed976', '#ffffcc', '#a1dab4', '#41b6c4', '#2c7fb8', '#253494'][:n_colors]

cmap = plt.get_cmap('RdYlGn_r')

# === Récupérer toutes les valeurs (pour une échelle cohérente)
all_values = []


for df in metrics_data.values():
    all_values.extend(df[metric].dropna().values)
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=cmap.N)

# === Création des sous-cartes
fig, axes = plt.subplots(1, 3, figsize=(22, 20), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0.12, bottom=0.4, top=0.98)

for ax, (name, df) in zip(axes, metrics_data.items()):
    africa_south.plot(ax=ax, color="whitesmoke", edgecolor="black")
    for label, shape in station_shapes.items():
        subset = df[df["label"] == label]
        ax.scatter(
            subset["longitude"], subset["latitude"],
            c=subset[metric], cmap=cmap, norm=norm,
            s=100, marker=shape, edgecolor="black"
        )
        # 🔁 Graduation personnalisée pour latitude
    lat_ticks = [-42, -35, -30, -25, -20, -15, -10]  # Valeurs en °S
    lat_labels = [f"{abs(val)}°S" for val in lat_ticks]
        # 🔁 Graduation personnalisée pour longitude
    lon_ticks = [10, 15, 20, 25, 30, 35, 40]  # Valeurs en °E
    lon_labels = [f"{val}°E" for val in lon_ticks]

    
    ax.set_title(f"{name}", fontsize=20)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    ax.set_xticklabels(lon_labels, fontsize=14)
    ax.set_yticklabels(lat_labels, fontsize=14)
    # Limiter l'affichage à la zone géographique ciblée
    ax.set_xlim(min(lon_ticks), max(lon_ticks))
    ax.set_ylim(min(lat_ticks), max(lat_ticks))
    ax.set_xlabel("Longitude", fontsize=16)
    ax.set_ylabel("Latitude", fontsize=16)

# ✅ Barre de couleurs
cbar_ax = fig.add_axes([0.2, 0.48, 0.5, 0.02])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                    cax=cbar_ax, orientation='horizontal')
cbar.set_label("Pearson_r [-]", loc='right', fontsize=16)
cbar.ax.xaxis.set_label_position('top') 
cbar.ax.tick_params(labelsize=14)  # ou 16, 18 selon ton besoin




    # 🔹 Légende des réseaux
legend_elements = [plt.Line2D([0], [0], marker=station_shapes[lab], color='black',
                                  linestyle='None', label=lab, markersize=14,
                                  markerfacecolor='white', markeredgecolor='black')
                       for lab in station_shapes]
ax.legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=14)


# Sauvegarde
save_path = r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\Compared_Dataset_Pearson_r_10m.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()




# === Métrique ciblée
metric = "PSS"
title = "Perkins Skill Score (PSS)"
# Plages de valeurs
bounds = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
n_colors = len(bounds) - 1

custom_colors = [
    '#f0f0f0',  # 0.0 : gris très clair
    '#d9d9d9',  # 0.1 : gris clair
    '#ffffcc',  # 0.2 : jaune pâle
    '#ffeda0',  # 0.3 : jaune doux
    '#fed976',  # 0.4 : jaune plus chaud
    '#c5dbf0',  # 0.5 : transition vers bleu clair
    '#9ecae1',  # 0.6 : bleu doux
    '#6baed6',  # 0.7 : bleu modéré
    '#3182bd',  # 0.8 : bleu foncé
    '#08519c'   # 0.9–1.0 : bleu profond
]

# Colormap personnalisée
cmap = mcolors.ListedColormap(custom_colors)

# === Récupérer toutes les valeurs (pour une échelle cohérente)
all_values = []


for df in metrics_data.values():
    all_values.extend(df[metric].dropna().values)
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=cmap.N)

# === Création des sous-cartes
fig, axes = plt.subplots(1, 3, figsize=(22, 20), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0.12, bottom=0.4, top=0.98)

for ax, (name, df) in zip(axes, metrics_data.items()):
    africa_south.plot(ax=ax, color="whitesmoke", edgecolor="black")
    for label, shape in station_shapes.items():
        subset = df[df["label"] == label]
        ax.scatter(
            subset["longitude"], subset["latitude"],
            c=subset[metric], cmap=cmap, norm=norm,
            s=100, marker=shape, edgecolor="black"
        )
        # 🔁 Graduation personnalisée pour latitude
    lat_ticks = [-42, -35, -30, -25, -20, -15, -10]  # Valeurs en °S
    lat_labels = [f"{abs(val)}°S" for val in lat_ticks]
        # 🔁 Graduation personnalisée pour longitude
    lon_ticks = [10, 15, 20, 25, 30, 35, 40]  # Valeurs en °E
    lon_labels = [f"{val}°E" for val in lon_ticks]

    
    ax.set_title(f"{name}", fontsize=20)
    ax.set_xticks(lon_ticks)
    ax.set_yticks(lat_ticks)
    ax.set_xticklabels(lon_labels, fontsize=14)
    ax.set_yticklabels(lat_labels, fontsize=14)
    # Limiter l'affichage à la zone géographique ciblée
    ax.set_xlim(min(lon_ticks), max(lon_ticks))
    ax.set_ylim(min(lat_ticks), max(lat_ticks))
    ax.set_xlabel("Longitude", fontsize=16)
    ax.set_ylabel("Latitude", fontsize=16)

# ✅ Barre de couleurs
cbar_ax = fig.add_axes([0.2, 0.48, 0.5, 0.02])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                    cax=cbar_ax, orientation='horizontal')
cbar.set_label("PSS [-]", loc='right', fontsize=16)
cbar.ax.xaxis.set_label_position('top') 
cbar.ax.tick_params(labelsize=14)  # ou 16, 18 selon ton besoin




    # 🔹 Légende des réseaux
legend_elements = [plt.Line2D([0], [0], marker=station_shapes[lab], color='black',
                                  linestyle='None', label=lab, markersize=14,
                                  markerfacecolor='white', markeredgecolor='black')
                       for lab in station_shapes]
ax.legend(handles=legend_elements, loc='lower right', frameon=True, fontsize=14)


# Sauvegarde
save_path = r"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\Compared_Dataset_PSS_10m.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 🔧 Dictionnaire des datasets
datasets = {
    "ICON-LAM": metrics_ICON_df.copy(),
    "ERA5": metrics_ERA5_df.copy(),
    "ERA5_GWA": metrics_ERA5_GWA_df.copy()
}

# 🔧 Palette et ordre
label_order = ["SASSCALWN", "TAHMO", "NCEI", "ALL"]
palette = {
    "SASSCALWN": "#66c2a5",
    "TAHMO": "gray",
    "NCEI": "#8da0cb",
    "ALL": "#e78ac3"
}

# 🔧 Métriques à tracer
metrics = {
    "MAE": "Mean Absolute Error (m/s)"
}

# 🔁 Boucle sur chaque métrique
for metric, metric_label in metrics.items():
    fig, axes = plt.subplots(1, 3, figsize=(21, 7), sharey=True)
    plt.suptitle(f"{metric_label} – Dataset vs Observed at 10m", fontsize=16, fontweight="bold")

    for ax, (ds_name, df) in zip(axes, datasets.items()):
        # 🔧 Ajouter catégorie ALL
        df_all = df.copy()
        df_all["label"] = "ALL"
        df_combined = pd.concat([df, df_all], ignore_index=True)

        # 📊 Boxplot
        sns.boxplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            palette=palette,
            showfliers=False,
            width=0.6,
            boxprops=dict(linewidth=1.2, edgecolor='black'),
            medianprops=dict(color="black", linewidth=2),
            ax=ax
        )

        # 📍 Points individuels
        sns.stripplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            color="orange", size=5, alpha=0.6,
            jitter=True,
            ax=ax
        )

        # 📍 Moyenne en ligne blanche pointillée + n
        for i, label in enumerate(label_order):
            vals = df_combined[df_combined["label"] == label][metric].dropna()
            if not vals.empty:
                mean_val = vals.mean()
                ax.hlines(mean_val, i - 0.2, i + 0.2, colors="white", linestyles="--", linewidth=2)
                ax.text(i, vals.max() + 0.05 * abs(vals.max()), f"n={len(vals)}", ha="center", fontsize=12)

        ax.set_title(ds_name, fontsize=18)
        ax.set_xlabel("")
        ax.set_ylabel("MAE(m/s)", fontsize=16)
        ax.set_xticklabels(label_order, rotation=20, fontsize=16)
        ax.tick_params(axis='y', labelsize=16)

    sns.despine()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"BoxPlot_GROUPED_10m_{metric.replace(' ', '_')}.png"
    save_path = rf"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\{filename}"
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()



# 🔧 Métriques à tracer
metrics = {
    "ME": "Mean Error (m/s)"
}

# 🔁 Boucle sur chaque métrique
for metric, metric_label in metrics.items():
    fig, axes = plt.subplots(1, 3, figsize=(21, 7), sharey=True)
    plt.suptitle(f"{metric_label} – Dataset vs Observed at 10m", fontsize=16, fontweight="bold")

    for ax, (ds_name, df) in zip(axes, datasets.items()):
        # 🔧 Ajouter catégorie ALL
        df_all = df.copy()
        df_all["label"] = "ALL"
        df_combined = pd.concat([df, df_all], ignore_index=True)

        # 📊 Boxplot
        sns.boxplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            palette=palette,
            showfliers=False,
            width=0.6,
            boxprops=dict(linewidth=1.2, edgecolor='black'),
            medianprops=dict(color="black", linewidth=2),
            ax=ax
        )

        # 📍 Points individuels
        sns.stripplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            color="orange", size=5, alpha=0.6,
            jitter=True,
            ax=ax
        )

        # 📍 Moyenne en ligne blanche pointillée + n
        for i, label in enumerate(label_order):
            vals = df_combined[df_combined["label"] == label][metric].dropna()
            if not vals.empty:
                mean_val = vals.mean()
                ax.hlines(mean_val, i - 0.2, i + 0.2, colors="white", linestyles="--", linewidth=2)
                ax.text(i, vals.max() + 0.05 * abs(vals.max()), f"n={len(vals)}", ha="center", fontsize=12)

        ax.set_title(ds_name, fontsize=18)
        ax.set_xlabel("")
        ax.set_ylabel("ME(m/s)", fontsize=16)
        ax.set_xticklabels(label_order, rotation=20, fontsize=16)
        ax.tick_params(axis='y', labelsize=16)

    sns.despine()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"BoxPlot_GROUPED_10m_{metric.replace(' ', '_')}.png"
    save_path = rf"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\{filename}"
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()



# 🔧 Métriques à tracer
metrics = {
    "Pearson r": "Pearson Correlation"
}

# 🔁 Boucle sur chaque métrique
for metric, metric_label in metrics.items():
    fig, axes = plt.subplots(1, 3, figsize=(21, 7), sharey=True)
    plt.suptitle(f"{metric_label} – Dataset vs Observed at 10m", fontsize=16, fontweight="bold")

    for ax, (ds_name, df) in zip(axes, datasets.items()):
        # 🔧 Ajouter catégorie ALL
        df_all = df.copy()
        df_all["label"] = "ALL"
        df_combined = pd.concat([df, df_all], ignore_index=True)

        # 📊 Boxplot
        sns.boxplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            palette=palette,
            showfliers=False,
            width=0.6,
            boxprops=dict(linewidth=1.2, edgecolor='black'),
            medianprops=dict(color="black", linewidth=2),
            ax=ax
        )

        # 📍 Points individuels
        sns.stripplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            color="orange", size=5, alpha=0.6,
            jitter=True,
            ax=ax
        )

        # 📍 Moyenne en ligne blanche pointillée + n
        for i, label in enumerate(label_order):
            vals = df_combined[df_combined["label"] == label][metric].dropna()
            if not vals.empty:
                mean_val = vals.mean()
                ax.hlines(mean_val, i - 0.2, i + 0.2, colors="white", linestyles="--", linewidth=2)
                ax.text(i, vals.max() + 0.05 * abs(vals.max()), f"n={len(vals)}", ha="center", fontsize=12)

        ax.set_title(ds_name, fontsize=18)
        ax.set_xlabel("")
        ax.set_ylabel("Pearson r[:]", fontsize=16)
        ax.set_xticklabels(label_order, rotation=20, fontsize=16)
        ax.tick_params(axis='y', labelsize=16)

    sns.despine()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"BoxPlot_GROUPED_10m_{metric.replace(' ', '_')}.png"
    save_path = rf"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\{filename}"
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()



# 🔧 Métriques à tracer
metrics = {
    "PSS": "Perkins Skill Score"
}

# 🔁 Boucle sur chaque métrique
for metric, metric_label in metrics.items():
    fig, axes = plt.subplots(1, 3, figsize=(21, 7), sharey=True)
    plt.suptitle(f"{metric_label} – Dataset vs Observed at 10m", fontsize=16, fontweight="bold")

    for ax, (ds_name, df) in zip(axes, datasets.items()):
        # 🔧 Ajouter catégorie ALL
        df_all = df.copy()
        df_all["label"] = "ALL"
        df_combined = pd.concat([df, df_all], ignore_index=True)

        # 📊 Boxplot
        sns.boxplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            palette=palette,
            showfliers=False,
            width=0.6,
            boxprops=dict(linewidth=1.2, edgecolor='black'),
            medianprops=dict(color="black", linewidth=2),
            ax=ax
        )

        # 📍 Points individuels
        sns.stripplot(
            data=df_combined,
            x="label", y=metric,
            order=label_order,
            color="orange", size=5, alpha=0.6,
            jitter=True,
            ax=ax
        )

        # 📍 Moyenne en ligne blanche pointillée + n
        for i, label in enumerate(label_order):
            vals = df_combined[df_combined["label"] == label][metric].dropna()
            if not vals.empty:
                mean_val = vals.mean()
                ax.hlines(mean_val, i - 0.2, i + 0.2, colors="white", linestyles="--", linewidth=2)
                ax.text(i, vals.max() + 0.05 * abs(vals.max()), f"n={len(vals)}", ha="center", fontsize=12)

        ax.set_title(ds_name, fontsize=18)
        ax.set_xlabel("")
        ax.set_ylabel("PSS[:]", fontsize=16)
        ax.set_xticklabels(label_order, rotation=20, fontsize=16)
        ax.tick_params(axis='y', labelsize=16)

    sns.despine()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"BoxPlot_GROUPED_10m_{metric.replace(' ', '_')}.png"
    save_path = rf"C:\Users\LENOVO\Downloads\pyton trainning\essais perso\week 16\Final coding\plot\10m\{filename}"
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()
